In [2]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [4]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# =============================
# Configuration
# =============================
EPOCHS = 10
BATCH_SIZE = 32
DATASET_DIR = "/kaggle/input/facedataset/Face Dataset/DataSet"

# =============================
# Load Data
# =============================
train_data = image_dataset_from_directory(
    DATASET_DIR,
    label_mode=None,
    image_size=(224, 224),
    batch_size=BATCH_SIZE
)

file_paths = train_data.file_paths

# Define label mappings
person_to_label = {'Muktadir': 0, 'Rahat': 1}
emotion_to_label = {'Angry': 0, 'Natural': 1, 'Smile': 2}

# =============================
# Label Extraction Functions
# =============================
def extract_labels(file_path):
    file_path_str = file_path.numpy().decode("utf-8")
    parts = file_path_str.split(os.path.sep)
    person_name = parts[-3]
    emotion_name = parts[-2]
    person_label = person_to_label[person_name]
    emotion_label = emotion_to_label[emotion_name]
    return np.int32(person_label), np.int32(emotion_label)

def get_labels(file_path):
    person, emotion = tf.py_function(func=extract_labels,
                                     inp=[file_path],
                                     Tout=[tf.int32, tf.int32])
    person.set_shape(())
    emotion.set_shape(())
    return person, emotion

file_path_ds = tf.data.Dataset.from_tensor_slices(file_paths)
label_ds = file_path_ds.map(get_labels, num_parallel_calls=tf.data.AUTOTUNE)
label_ds = label_ds.batch(BATCH_SIZE)
train_ds = tf.data.Dataset.zip((train_data, label_ds)).prefetch(tf.data.AUTOTUNE)

# =============================
# Load MobileNetV2 Model
# =============================
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

# Two outputs
person_output = Dense(2, activation='softmax', name='person_output')(x)
emotion_output = Dense(3, activation='softmax', name='emotion_output')(x)

model = Model(inputs=base_model.input, outputs=[person_output, emotion_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'person_output': 'sparse_categorical_crossentropy',
        'emotion_output': 'sparse_categorical_crossentropy'
    },
    metrics={
        'person_output': 'accuracy',
        'emotion_output': 'accuracy'
    }
)

model.summary()

# =============================
# Train Model
# =============================
history = model.fit(train_ds, epochs=EPOCHS)

# Save the trained model
model.save("face_emotion_mobilenet2.h5")
print("MobileNetV2-based model saved successfully.")

Found 0 files.


ValueError: No images found in directory /kaggle/input/facedataset/Face Dataset/DataSet. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')